In [1]:
# fine tune the model on the CoT dataset
# train only for output tokens in the dataset

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, TrainerCallback
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model
import wandb


peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)


batch_size = 8

model_name = "facebook/galactica-125m"
tokenizer_name = model_name

dataset_name = "jeggers/CoT-Collection-finetuning"
input_column = "final_input"
output_column = "final_target"

c:\Users\jorin\.conda\envs\AGI\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
# load model, tokenizer and dataset
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")
dataset = load_dataset(dataset_name)
dataset = dataset["train"].train_test_split(test_size=0.1)

# create new 'all' column that contains both input and output tokens
def combine_input_output(batch):
    return {"all": [x + y + "</s>" for x, y in zip(batch[input_column], batch[output_column])]}
dataset = dataset.map(combine_input_output, batched=True, batch_size=-1)

# convert to peft model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001FFFBA033D0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 20238f0f8d0, raw_cell="# load model, tokenizer and dataset
tokenizer = Au.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/d%3A/Google%20Drive/Other%20computers/My%20laptop/Daten/Hobbys/Informatik/Mashine%20leaning/ThinkingLLMs/finetune_model.ipynb#W1sZmlsZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

c:\Users\jorin\.conda\envs\AGI\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 145/145 [00:00<00:00, 5072.18 examples/s]

trainable params: 294,912 || all params: 125,325,312 || trainable%: 0.2353
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001FFFBA033D0>> (for post_run_cell), with arguments args (<ExecutionResult object at 2024436fb10, execution_count=33 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 20238f0f8d0, raw_cell="# load model, tokenizer and dataset
tokenizer = Au.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/d%3A/Google%20Drive/Other%20computers/My%20laptop/Daten/Hobbys/Informatik/Mashine%20leaning/ThinkingLLMs/finetune_model.ipynb#W1sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [34]:
# print first 5 examples
from pprint import pprint
pprint(dataset["train"]["all"][:5])

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001FFFBA033D0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 2024435fe50, raw_cell="# print first 5 examples
from pprint import pprint.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/d%3A/Google%20Drive/Other%20computers/My%20laptop/Daten/Hobbys/Informatik/Mashine%20leaning/ThinkingLLMs/finetune_model.ipynb#X11sZmlsZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

['Below are two sentences. Identify the word that appears in both and decide '
 "if its meaning is consistent across the sentences. Answer 'True' or "
 "'False'.\n"
 '\n'
 'Sentence 1: Can you play on this old recorder?\n'
 '\n'
 'Sentence 2: He plays the flute.\n'
 'BOT: ANSWER: True</s>',
 'When the division was calculated by changing the tens and units digits of '
 'the divisor, the quotient and remainder were 25 each. When the divisor is '
 '65, find the sum of the correctly calculated quotient and remainder.\n'
 'BOT: 1. The original divisor had its digits swapped, so it was 56.\n'
 '2. With 56 as divisor, quotient and remainder were both 25.\n'
 '3. This means the dividend was 56 * 25 + 25 = 1425.\n'
 '4. Now, we need to divide 1425 by 65 (the correct divisor).\n'
 '5. 1425 ÷ 65 = 21 remainder 60\n'
 '6. Sum of correct quotient and remainder: 21 + 60 = 81\n'
 '\n'
 'ANSWER: 81</s>',
 'Yoongi got 4, and Yuna got 5, and Jungkook got the numbr that is 6 '
 'multiplied by 3. Who got 

TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [35]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = 1
if tokenizer.eos_token_id is None:
    tokenizer.eos_token_id = 2

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001FFFBA033D0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1fffba3d810, raw_cell="if tokenizer.pad_token_id is None:
    tokenizer.p.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/d%3A/Google%20Drive/Other%20computers/My%20laptop/Daten/Hobbys/Informatik/Mashine%20leaning/ThinkingLLMs/finetune_model.ipynb#W3sZmlsZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001FFFBA033D0>> (for post_run_cell), with arguments args (<ExecutionResult object at 20238dee810, execution_count=35 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1fffba3d810, raw_cell="if tokenizer.pad_token_id is None:
    tokenizer.p.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/d%3A/Google%20Drive/Other%20computers/My%20laptop/Daten/Hobbys/Informatik/Mashine%20leaning/ThinkingLLMs/finetune_model.ipynb#W3sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [37]:
# tokenize the dataset
def tokenize_function(examples):
    examples = tokenizer(examples['all'], padding="max_length", max_length=512, return_tensors="pt")
    examples["labels"] = examples["input_ids"].clone()
    return examples

tokenized_datasets = dataset.map(tokenize_function, batched=True, batch_size=batch_size, drop_last_batch=True)

tokenized_datasets


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001FFFBA033D0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 20244367d90, raw_cell="# tokenize the dataset
def tokenize_function(examp.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/d%3A/Google%20Drive/Other%20computers/My%20laptop/Daten/Hobbys/Informatik/Mashine%20leaning/ThinkingLLMs/finetune_model.ipynb#W6sZmlsZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Map: 100%|██████████| 144/144 [00:00<00:00, 957.56 examples/s] 


DatasetDict({
    train: Dataset({
        features: ['final_input', 'final_target', 'all', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1296
    })
    test: Dataset({
        features: ['final_input', 'final_target', 'all', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 144
    })
})

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001FFFBA033D0>> (for post_run_cell), with arguments args (<ExecutionResult object at 20245910f90, execution_count=37 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 20244367d90, raw_cell="# tokenize the dataset
def tokenize_function(examp.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/d%3A/Google%20Drive/Other%20computers/My%20laptop/Daten/Hobbys/Informatik/Mashine%20leaning/ThinkingLLMs/finetune_model.ipynb#W6sZmlsZQ%3D%3D> result=DatasetDict({
    train: Dataset({
        features: ['final_input', 'final_target', 'all', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1296
    })
    test: Dataset({
        features: ['final_input', 'final_target', 'all', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 144
    })
})>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [38]:
# print tensor of first sample
print(tokenized_datasets["train"]["input_ids"][3])

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001FFFBA033D0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 202443679d0, raw_cell="# print tensor of first sample
print(tokenized_dat.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/d%3A/Google%20Drive/Other%20computers/My%20laptop/Daten/Hobbys/Informatik/Mashine%20leaning/ThinkingLLMs/finetune_model.ipynb#X21sZmlsZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

[5492, 286, 1179, 7378, 34, 6692, 286, 2882, 404, 897, 2224, 363, 286, 1806, 11905, 36, 221, 221, 9357, 48, 221, 40593, 107, 2947, 417, 2801, 243, 24, 831, 39116, 8598, 299, 286, 6222, 24, 1543, 1652, 1574, 3086, 11262, 312, 1024, 817, 4972, 301, 41681, 36, 22381, 682, 1997, 299, 41681, 2095, 495, 1673, 1101, 53, 221, 221, 22126, 48, 221, 55, 48, 10281, 13689, 312, 281, 14043, 7291, 4310, 221, 56, 48, 9366, 34508, 312, 10281, 13689, 221, 57, 48, 281, 14043, 7291, 4310, 312, 10311, 1484, 221, 58, 48, 10311, 1484, 312, 9366, 34508, 221, 56, 4675, 48, 2844, 11677, 1409, 48, 410, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [6]:
inputs = tokenizer("Hello, how are you?", return_tensors="pt").to("cuda")
labels = inputs["input_ids"].clone()
del inputs["token_type_ids"]

outputs = model(**inputs, labels=labels)
print(outputs.loss)

tensor(8.3033, device='cuda:0', grad_fn=<NllLossBackward0>)


In [7]:
# collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

class LoggingCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if "loss" in logs:
            wandb.log({"training_loss": logs["loss"]})
        if "eval_loss" in logs:
            wandb.log({"eval_loss": logs["eval_loss"]})

training_args = TrainingArguments(
    output_dir="your-name/bigscience/mt0-large-lora",
    learning_rate=1e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    logging_strategy="steps",
    logging_steps=1,
    load_best_model_at_end=True,
    report_to="wandb",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    callbacks=[LoggingCallback()],
    # data_collator=collator,
    # compute_metrics=compute_metrics,
)


In [8]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jorineggers. Use `wandb login --relogin` to force relogin


  0%|          | 1/324 [00:10<54:12, 10.07s/it]

{'loss': 15.385, 'grad_norm': 14.416722297668457, 'learning_rate': 0.0009969135802469135, 'epoch': 0.01}


  1%|          | 2/324 [00:14<35:31,  6.62s/it]

{'loss': 12.0557, 'grad_norm': 16.250343322753906, 'learning_rate': 0.000993827160493827, 'epoch': 0.01}


...

100%|██████████| 324/324 [24:10<00:00,  4.48s/it]

{'loss': 3.2056, 'grad_norm': 0.46716541051864624, 'learning_rate': 0.0, 'epoch': 2.0}
{'train_runtime': 1463.4788, 'train_samples_per_second': 1.771, 'train_steps_per_second': 0.221, 'train_loss': 3.403447809778614, 'epoch': 2.0}


TrainOutput(global_step=324, training_loss=3.403447809778614, metrics={'train_runtime': 1463.4788, 'train_samples_per_second': 1.771, 'train_steps_per_second': 0.221, 'total_flos': 679617220313088.0, 'train_loss': 3.403447809778614, 'epoch': 2.0})

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001FFFBA033D0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1fff9abdb50, execution_count=8 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1fff9abdb90, raw_cell="trainer.train()" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/d%3A/Google%20Drive/Other%20computers/My%20laptop/Daten/Hobbys/Informatik/Mashine%20leaning/ThinkingLLMs/finetune_model.ipynb#X12sZmlsZQ%3D%3D> result=TrainOutput(global_step=324, training_loss=3.403447809778614, metrics={'train_runtime': 1463.4788, 'train_samples_per_second': 1.771, 'train_steps_per_second': 0.221, 'total_flos': 679617220313088.0, 'train_loss': 3.403447809778614, 'epoch': 2.0})>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [9]:
# merge adapter weights into model
model = model.merge_and_unload()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001FFFBA033D0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1fffba6bb10, raw_cell="# merge adapter weights into model
model = model.m.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/d%3A/Google%20Drive/Other%20computers/My%20laptop/Daten/Hobbys/Informatik/Mashine%20leaning/ThinkingLLMs/finetune_model.ipynb#X14sZmlsZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001FFFBA033D0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1fff9a609d0, execution_count=9 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1fffba6bb10, raw_cell="# merge adapter weights into model
model = model.m.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/d%3A/Google%20Drive/Other%20computers/My%20laptop/Daten/Hobbys/Informatik/Mashine%20leaning/ThinkingLLMs/finetune_model.ipynb#X14sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [10]:
model.push_to_hub("jeggers/galactica-125m-cot")

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001FFFBA033D0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 1fffba03210, raw_cell="model.push_to_hub("jeggers/galactica-125m-cot")" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/d%3A/Google%20Drive/Other%20computers/My%20laptop/Daten/Hobbys/Informatik/Mashine%20leaning/ThinkingLLMs/finetune_model.ipynb#X15sZmlsZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

model.safetensors: 100%|██████████| 500M/500M [01:50<00:00, 4.54MB/s]   


CommitInfo(commit_url='https://huggingface.co/jeggers/galactica-125m-cot/commit/0389aaf062cb8360ebb19f26bf52b16cabaeefff', commit_message='Upload OPTForCausalLM', commit_description='', oid='0389aaf062cb8360ebb19f26bf52b16cabaeefff', pr_url=None, pr_revision=None, pr_num=None)

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001FFFBA033D0>> (for post_run_cell), with arguments args (<ExecutionResult object at 1fff9b03950, execution_count=10 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 1fffba03210, raw_cell="model.push_to_hub("jeggers/galactica-125m-cot")" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/d%3A/Google%20Drive/Other%20computers/My%20laptop/Daten/Hobbys/Informatik/Mashine%20leaning/ThinkingLLMs/finetune_model.ipynb#X15sZmlsZQ%3D%3D> result=CommitInfo(commit_url='https://huggingface.co/jeggers/galactica-125m-cot/commit/0389aaf062cb8360ebb19f26bf52b16cabaeefff', commit_message='Upload OPTForCausalLM', commit_description='', oid='0389aaf062cb8360ebb19f26bf52b16cabaeefff', pr_url=None, pr_revision=None, pr_num=None)>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [17]:
tokenizer.push_to_hub("jeggers/galactica-125m-cot")

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001FFFBA033D0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 202387f65d0, raw_cell="tokenizer.push_to_hub("jeggers/galactica-125m-cot".." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/d%3A/Google%20Drive/Other%20computers/My%20laptop/Daten/Hobbys/Informatik/Mashine%20leaning/ThinkingLLMs/finetune_model.ipynb#X20sZmlsZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

c:\Users\jorin\.conda\envs\AGI\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jorin\.cache\huggingface\hub\models--jeggers--galactica-125m-cot. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/jeggers/galactica-125m-cot/commit/19b2769c56fc4eef8d1aab2dfaf2a7cf74bb6b3c', commit_message='Upload tokenizer', commit_description='', oid='19b2769c56fc4eef8d1aab2dfaf2a7cf74bb6b3c', pr_url=None, pr_revision=None, pr_num=None)

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001FFFBA033D0>> (for post_run_cell), with arguments args (<ExecutionResult object at 20169d3af90, execution_count=17 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 202387f65d0, raw_cell="tokenizer.push_to_hub("jeggers/galactica-125m-cot".." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/d%3A/Google%20Drive/Other%20computers/My%20laptop/Daten/Hobbys/Informatik/Mashine%20leaning/ThinkingLLMs/finetune_model.ipynb#X20sZmlsZQ%3D%3D> result=CommitInfo(commit_url='https://huggingface.co/jeggers/galactica-125m-cot/commit/19b2769c56fc4eef8d1aab2dfaf2a7cf74bb6b3c', commit_message='Upload tokenizer', commit_description='', oid='19b2769c56fc4eef8d1aab2dfaf2a7cf74bb6b3c', pr_url=None, pr_revision=None, pr_num=None)>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [28]:
question = "Pick the optimal word to complete the phrase: Natalie was trying to fit a prom dress over their client's bust, but struggled to do so because the _ was too big. A: dress B: bust Provide your answer using the letter only."
inputs = tokenizer(question + " BOT: ", return_tensors="pt").to("cuda")
del inputs["token_type_ids"]
out = model.generate(**inputs, max_new_tokens=120, num_return_sequences=1, do_sample=True, temperature=0.8)
text = tokenizer.batch_decode(out, skip_special_tokens=True)
print(text)
print(out)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000001FFFBA033D0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 20238e9c250, raw_cell="question = "Pick the optimal word to complete the .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/d%3A/Google%20Drive/Other%20computers/My%20laptop/Daten/Hobbys/Informatik/Mashine%20leaning/ThinkingLLMs/finetune_model.ipynb#X16sZmlsZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

['Pick the optimal word to complete the phrase: Natalie was trying to fit a prom dress over their client\'s bust, but struggled to do so because the _ was too big. A: dress B: bust Provide your answer using the letter only. BOT: 1. Attend to the phrase: Natalie was trying to fit a prom dress over her client\'s bust.\n2. As the question shows, the word and sentence are separated, the first sentence is a word of length 2.\n3. The sentence is meant to complete the phrase: "Try to work out a prom dress on the client\'s bust."\n4. The word "B" must refer to the large bust.\n5. The sentence points to a large bust.\n\nANSWER: B� expendit']
tensor([[32893,   286,  2936,  5104,   321,  2856,   286, 20459,    48, 38703,
          1069,   435, 13571,   321,  2422,   281,  1928, 35998,   797,   817,
         10753,    29,   105,   311,   690,    34,   835, 20633,  1663,   321,
           917,   891,  1543,   286,   243,    85,   435,  4606,  6499,    36,
           351,    48, 35998,   410,    48,

TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given